In [1]:
using Pkg
Pkg.instantiate()
using HTTP, JSON, JLD, Roots, PrettyTables, DotEnv
DEBUG_LEVEL = 1
using Optim, Random
Random.seed!(0)
using BlackBoxOptim, Distributions, ForwardDiff, Integrals, Roots, StatsPlots, DelimitedFiles
Pkg.instantiate()
cfg = DotEnv.config("../.env")
files_path = cfg["files_path"]

┌ Warning: Error requiring `HTTP` from `BlackBoxOptim`
│   exception =
│    LoadError: ArgumentError: Package BlackBoxOptim does not have HTTP in its dependencies:
│    - You may have a partially installed environment. Try `Pkg.instantiate()`
│      to ensure all packages in the environment are installed.
│    - Or, if you have BlackBoxOptim checked out for development and have
│      added HTTP as a dependency but haven't updated your primary
│      environment's manifest file, try `Pkg.resolve()`.
│    - Otherwise you may need to report an issue with BlackBoxOptim
│    Stacktrace:
│      [1] macro expansion
│        @ ./loading.jl:1167 [inlined]
│      [2] macro expansion
│        @ ./lock.jl:223 [inlined]
│      [3] require(into::Module, mod::Symbol)
│        @ Base ./loading.jl:1144
│      [4] include(mod::Module, _path::String)
│        @ Base ./Base.jl:419
│      [5] include(x::String)
│        @ BlackBoxOptim ~/.julia/packages/BlackBoxOptim/lZtsr/src/BlackBoxOptim.jl:1
│      [6

"/home/peters/code/mapinator/estimation/current_estimates_and_files/"

In [2]:
# change associated with ocs as failed to trade group
raw_placement_rates = load(files_path*"adjusted_placement_rates.jld")["adjusted_placement_rates"]
unmatched_row_index = 10
adjusted_placement_rates = vcat(raw_placement_rates[1:unmatched_row_index-1,:],
raw_placement_rates[unmatched_row_index+1:size(raw_placement_rates)[1],:])
#adjusted_placement_rates = load(files_path*"adjusted_placement_rates.jld")["adjusted_placement_rates"]
NUMBER_OF_TYPES = size(adjusted_placement_rates, 2)

adjusted_placement_rates = vcat(adjusted_placement_rates, raw_placement_rates[unmatched_row_index:unmatched_row_index, :])
numtotal = size(adjusted_placement_rates, 1)

12

In [3]:
#setup stuff
k = NUMBER_OF_TYPES
#since one row is unmatched
K = numtotal
M = sum(adjusted_placement_rates)
#maximum likelihood
L = true
#chi-squared
#L = false

# upper bound on the value ratios, which should all be less than 1
# if any ratio turns out to be 1.0 or close to it at optimality, this could indicate that a lower tier has a higher value than a higher one
upper = [1.0 for _ in 1:k-1] 

# upper bound on variables proportionate to alpha
append!(upper, [1.0 for _ in 1:k])

# upper bound on the mu parameter of truncated normal, which is strictly within [0, 1] as the mean is greater than mu in truncated normal
append!(upper, [1.0 for _ in 1:K])

# upper bound on the sigma parameter of truncated normal
append!(upper, [5.0 for _ in 1:K])

# upper bound on variables proportionate to rho
append!(upper, [1.0 for _ in 1:K])

# upper bound on beta friction parameter
push!(upper, 1.0);


In [4]:
function F(x, ρ, μ, σ, K)
    return sum([ρ[i] * cdf(truncated(Normal(μ[i], σ[i]), 0, 1), x) for i in 1:K])
end

function f(x, ρ, μ, σ, K)
    return sum([ρ[i] * pdf(truncated(Normal(μ[i], σ[i]), 0, 1), x) for i in 1:K])
end

function G(Fim1, Fx, αsum)
    return (Fim1 - Fx) / αsum
end

function κ(i, t, v_rel)
    return sum([-log(v_rel[j]) for j in t:i-1])
end

function fi(x, μ, σ)
    return pdf(truncated(Normal(μ, σ), 0, 1), x)
end

fi (generic function with 1 method)

In [5]:
function q(i, t, Fx_vec, x_vec, ρ, μ, σ, α, v_rel, k, K)
    # Fx_vec = [F(x0)=1, F(x1), F(x2), F(x3), ..., F(xk-1)]
    # x_vec = [x0 = 1, x1, x2, x3, ..., xk = 0]
    # x_vec[s] = x_{s-1}, so the limits of integration are and must be offset by 1 below
    # TODO: can some integrals be cached as a speed-up? can some integrals be computed in parallel?
    return sum([(α[t]/sum(α[1:s])) * 
            solve(IntegralProblem{false}((x, p) -> exp(-(G(Fx_vec[s], F(x, ρ, μ, σ, K), sum(α[1:s])) + κ(s, t, v_rel))) *
                    fi(x, μ[i], σ[i]), x_vec[s+1], x_vec[s]), HCubatureJL())[1] for s in t:k])
end

function Fx(t, α, v_rel)
    return 1 - sum([-log(v_rel[j])*sum(α[1:j]) for j in 1:t])
end

function Q2(ratio, β)
    return β * (1 - exp(-ratio)) / ratio
end

function pi(t, α)
    return α[t] / sum(α[1:t])
end

pi (generic function with 1 method)

In [6]:
"""
Likelihood
    p_vec -> k values,k alpha shares, K means, K variances, K beliefs
    example -  for 5 academic tiers and six hiring tiers p_vec has 43 elements
    - it assumes the last row of the adjacency matrix is for unmatched so the placement matrix in the example has dimension 12x5 not 11x5
    placements -> the adjacency matrix with the unmatched row moved to the bottom
    k -> number of academic tiers
    K -> rows in the adjacency matrix - last row is treaed as unmatched
    L -> true for likelihood, false for chi-squared maximization
"""
function estimate_likelihood(p_vec, placements, k, K, M, L)
    #values
    v_rel = p_vec[1:k-1]
    #shares
    α = 1.0 * p_vec[k:2k-1] / sum(p_vec[k:2k-1]) # change 1.0 to scale number of graduates vs departments
    #means
    μ = p_vec[2k:2k+K-2]
    #variances
    σ = p_vec[2k+K-1:2k+2K-3]
    #beliefs renormalized
    ρ = p_vec[2k+2K-2:2k+3K-4] / sum(p_vec[2k+2K-2:2k+3K-4])
    #β = p_vec[end] # set = 1 to hardcode

    Fx_vec = ones(k)
    x_vec = ones(k+1)
    x_vec[k+1] = 0.0
    for t in 1:k-1
        Fx_vec_candidate = Fx(t, α, v_rel)
        if Fx_vec_candidate <= 0.0
            Fx_vec[t+1:k] .= 0.0
            x_vec[t+1:k] .= 0.0
            break
        end
        Fx_vec[t+1] = Fx_vec_candidate
        x_vec[t+1] = find_zero(x -> F(x, ρ, μ, σ, K) - Fx_vec[t+1], 0.5) 
    end

    objective = 0.0
    likelihood = 0.0
    normalizer= zeros(K)
    # recode
    # 1-normalizer is now the probabiity a placement should end up in ocs  
    q_it = zeros(K, k)
    for i in 1:K, t in 1:k
        prob = q(i, t, Fx_vec, x_vec, ρ, μ, σ, α, v_rel, k, K)
        q_it[i, t] = prob
        normalizer[i] += ρ[i] * prob
    end
    for i in 1:K-1, t in 1:k 
        expectation = M * (ρ[i] * (q_it[i, t]))
        if L
            objective += (placements[i, t] - expectation) ^ 2 / expectation
        else
            # likilihood
            objective += placements[i, t] * (log(ρ[i] * (q_it[i, t] )))
            objective -= log(factorial(big(placements[i, t])))
        end
    end
    for t in 1:k
        expectation = M*(1-normalizer[t])
        if L
            objective += (placements[K, t] - expectation)^2 / expectation
        else
            #likelihood
            objective += placements[K, t]*log(1 - normalizer[t])
            objective -= log(factorial(big(placements[K, t])))
        end
    end
    return -objective
end


estimate_likelihood

In [7]:
"""
    sol -> the solution provided by bboptimize
    placements ->   the actual placements including unmatched dimensions computer from the size of this matrix
"""
function print_metrics_maximum_likelihood(sol, placements) 
    k = size(placements)[1]
    K = size(placements)[2]
    M = sum(placements)
    println("chi-square objective value = ", objective)
    println("log-likelihood objective value = ", likelihood)
    println("likelihood objective value = ", exp(likelihood))
    println("success sample size (departments) = ", M)
    #println("estimated total samples (departments) = ", M / normalizer)
    #println("estimated unmatched departments = ", (M / normalizer) - M)
    #println("probability of any success: ", normalizer)
    #println("probability of no success: ", 1 - normalizer)
    #println("measure of departments in round 2 = ", D)
    #println("measure of graduates in round 2 = ", S)
    #println()
    println("predicted fraction of departments of each tier:")
    display(ρ)
    println()
    println("fractions observed among successful departments in data:")
    display(sum(placements, dims = 2) ./ M)
    println()

    for i in 1:k
        println("pi_", i, " = ", pi(i, α))
    end
    println()

    offer_targets = zeros(k, k)
    for t in 1:k, j in 1:t
        offer_targets[t, j] = pi(j, α) * prod([1 - pi(i, α) for i in j+1:t])
    end
    println("Tier selection probabilities for making offers:")
    display(offer_targets)
    println()

    println("Round 1 hiring probabilities:")
    display(q_it)
    println()

    
    for i in 1:k+1
        println("x_", i - 1, " = ", x_vec[i])
    end
    println()
    for i in 1:k
        println("F(x_", i - 1, ") = ", Fx_vec[i])
    end
    println()
    #for i in 1:k
    #    println("α_", i, " = ", α[i])
    #    println("  Est. graduates: ", α[i] * (M / normalizer - 1))
    #    println("  Successful: ", sum(placements[:, i]))
    #    println("  Unsuccessful: ", (α[i] * (M / normalizer - 1)) - sum(placements[:, i]))
    #end
    #println("Total estimated graduates: ", sum(α) * (M / normalizer - 1))
    println("Total successful graduates: ", M)
    #println("Total estimated unsuccessful graduates: ", (sum(α) * (M / normalizer - 1)) - M)
    #println("β = ", β)
    println()
    println("actual placement rates:")
    display(placements)
    println()
    #println("difference between estimated and actual placement rates:")
    #display(exp_placements - placements)
    #println()
    println("chi-square p-value")
    println(1 - cdf(Chisq((size(placements)[1] - 1) * (size(placements)[2] - 1)), objective))
end

print_metrics_maximum_likelihood

In [8]:

# all lower bounds are zero as these should be positive parameters
# can swap estimate_likelihood for chi_square
sol_res = bboptimize(p -> estimate_likelihood(p, adjusted_placement_rates, k, K, M, L),
    SearchRange = [(0.0, upper[i]) for i in eachindex(upper)], MaxFuncEvals = 100000, TraceInterval = 5)
sol = best_candidate(sol_res)

LoadError: BoundsError: attempt to access 11-element Vector{Float64} at index [12]

In [9]:
# uncomment this after estimation if you want detailed estimates

# ensure the correct metrics function is used based on the selected optimiz
estimate_likelihood(sol,adjusted_placement_rates, 5,11,sum(adjusted_placement_rates),true)
#print_metrics_maximum_likelihood(sol, adjusted_placement_rates)
size(sol)

LoadError: UndefVarError: sol not defined

In [10]:
v_base = 1
type_values = []
type_headers = []
row_names = ["Graduate Values"]
for i in 1:k
 #   println("v", i, ": ", v_base)
    push!(type_values, v_base)
    push!(type_headers, string("Tier ", i))
    if i != k
        v_base = sol[i] * v_base
    end
end
data = Vector{Float64}(type_values)
head = Vector{String}(type_headers)
#println(size(data,2), " " , size(data,1))
#println(Vector{String}(type_headers))
#println(row_names)
#pretty_table(data', header = head, row_names=row_names)
means_by_type = []
std_by_type = []
for select_type in 1:k
    push!(means_by_type, mean(truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0 ,1)))
    push!(std_by_type, std(truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0 ,1)))
   
    #println("mean for type ", select_type, ": ", mean(truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0, 1)))
    #println("stddev for type ", select_type, ": ", std(truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0, 1)))
    #println()
end
datam = Vector{Float64}(means_by_type)
datas = Vector{Float64}(std_by_type)
shares = 1.0 * sol[k:2k-1] / sum(sol[k:2k-1]) # change 1.0 to scale number of graduates vs departments
all_data = hcat(data, datam, datas, shares )
push!(row_names, "Mean offer value")
push!(row_names, "Offers, std deviation")
push!(row_names, "Tier Shares")

#println(all_data)    
pretty_table(all_data', header = head, row_labels=row_names)

#for select_type in k+1:K
#    println("mean for sink ", select_type - k, ": ", mean(truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0, 1)))
#    println("stddev for sink ", select_type - k, ": ", std(truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0, 1)))
#    println()
#end

LoadError: UndefVarError: sol not defined

In [11]:
save(files_path*"current_estimates.jld", "all_data", all_data)

LoadError: UndefVarError: all_data not defined

In [12]:
open(files_path*"current_estimates.tex", "w") do f
pretty_table(
    f,
    all_data',
    header = head,
        row_labels = row_names,
    backend = Val(:latex)
    )
end

LoadError: UndefVarError: all_data not defined

In [13]:
# https://github.com/JuliaPlots/StatsPlots.jl/blob/master/README.md
# https://docs.juliaplots.org/latest/tutorial/

select_type = 1
cdfs = plot(truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0, 1), func = cdf, title = "CDFs of Types", label = "Type 1")
for select_type in 2:NUMBER_OF_TYPES # academic types
    plot!(cdfs, truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0, 1), func = cdf, label = string("Type ", select_type))
end

for select_type in k+1:K # sinks
    plot!(cdfs, truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0, 1), func = cdf, label = string("Sink ", select_type - k))
end
xlabel!(cdfs, "offer value")
ylabel!(cdfs, "F(offer value)")
savefig(cdfs, "cdfs.png")
cdfs

LoadError: UndefVarError: sol not defined

In [14]:
expected_offers = []
for select_type in 1:K
     push!(expected_offers,mean(truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0, 1)))
end
expected_offers
save(files_path*"expected_offers.jld", "expected_offers", expected_offers)

LoadError: UndefVarError: sol not defined

In [15]:
select_type = 1
pdfs = plot(truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0, 1), func = pdf, title = "PDFs of Types", label = "Type 1")
for select_type in 2:k # academic types
    plot!(pdfs, truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0, 1), func = pdf, 
        label = string("Type ", select_type))
end

for select_type in k+1:K # sinks
    plot!(pdfs, truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0, 1), func = pdf, label = string("Sink ", select_type - k))
end
xlabel!(pdfs, "offer value")
ylabel!(pdfs, "f(offer value)")
savefig(pdfs, "pdfs.png")
pdfs

LoadError: UndefVarError: sol not defined